In [86]:
pip install fuzzywuzzy

Note: you may need to restart the kernel to use updated packages.


In [87]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [88]:
members = pd.read_csv('./data/cleaned_members.csv')

In [89]:
main = pd.read_csv('./data/senate_and_house.csv')

In [90]:
members.head()

,people,state,party,total_senate_years,total_house_years,total_years
0,James Abdnor,South Dakota,Republican,6,8,14
1,Neil Abercrombie,Hawaii,Democratic,0,22,22
2,James Abourezk,South Dakota,Democratic,6,2,8
3,"Ralph Abraham, Lee",Louisiana,Republican,0,6,6
4,Spencer Abraham,Michigan,Republican,6,0,6


In [91]:
main.head()

,Unnamed: 0,transaction_date,ticker,asset_description,type,amount,representative
0,4,2020-03-18,CBRE,CBRE Group Inc Common Stock Class A,sale_partial,"$1,001 - $15,000",Susie Lee
1,5,2020-03-24,CNC,Centene Corporation,purchase,"$1,001 - $15,000",Susie Lee
2,6,2020-03-31,DG,Dollar General Corporation,purchase,"$1,001 - $15,000",Susie Lee
3,10,2020-02-21,ENV,"Envestnet, Inc",sale_partial,"$1,001 - $15,000",Susie Lee
4,12,2020-03-17,LKQ,LKQ Corporation,sale_partial,"$1,001 - $15,000",Susie Lee


In [92]:
# def fuzzy_merge(main, members, representative, people, threshold=90, limit=2):
#     """
#     :param df_1: the left table to join
#     :param df_2: the right table to join
#     :param key1: key column of the left table
#     :param key2: key column of the right table
#     :param threshold: how close the matches should be to return a match, based on Levenshtein distance
#     :param limit: the amount of matches that will get returned, these are sorted high to low
#     :return: dataframe with boths keys and matches
#     """
#     s = members['people'].tolist()
    
#     m = main['representative'].apply(lambda x: process.extract(x, s, limit=limit))    
#     main['matches'] = m
    
#     m2 = main['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
#     main['matches'] = m2
    
#     return main

In [93]:
# unique in main
names_in_main = main.representative.unique()

# made into a list 
names = []
for name in names_in_main:
    names.append(name)
type(names)

#list of members df 
member_names = []
for name in members.people.values:
    member_names.append(name)

In [94]:
len(member_names)

2422

In [95]:
# unique names in main
keys = {}

for name in names:
    #finding closest math to main data from larger 'members data'
    keys[name] = ((process.extract(name, member_names, limit=2)))

In [96]:
keys

{'Susie Lee': [('Susie Lee', 100), ('Ralph Abraham, Lee', 86)],
 'Alan S. Lowenthal': [('Alan Lowenthal, S.', 95), ('Alan Bible', 86)],
 'Mo Brooks': [('Mo Brooks', 100), ('Jack Brooks, B.', 86)],
 'Robert J. Wittman': [('Robert Wittman, J.', 95), ('J. Beall', 86)],
 'Lois Frankel': [('Lois Frankel', 100), ('Al Franken', 73)],
 'Michael T. McCaul': [('Michael McCaul, T.', 95),
  ('Michael McNulty, R.', 75)],
 'Suzan K. DelBene': [('Suzan DelBene, K.', 95), ('John Delaney, K.', 59)],
 'Greg Gianforte': [('Greg Gianforte', 100), ('Greg Ganske', 72)],
 'Lloyd K. Smucker': [('Lloyd Smucker', 95), ('Floyd Haskell, K.', 64)],
 'Earl Blumenauer': [('Earl Blumenauer', 100), ('Earl Hutto', 86)],
 'James Comer': [('James Comer', 100), ('James Broyhill, T.', 86)],
 'James R. Langevin': [('James Langevin, R.', 95), ('James Comer', 86)],
 'Anthony E. Gonzalez': [('Anthony Gonzalez', 95), ('E. Garn', 86)],
 'Josh Gottheimer': [('Josh Gottheimer', 100), ('Joshua Eilberg', 62)],
 'Katherine M. Clark':

In [97]:
#where scores live
# for i in range(1, 173):
#     print(keys[i][1][1])

In [100]:
merge_key = [keys[name][0][0] for name in main['representative']]

In [101]:
main['merge_key'] = merge_key

In [103]:
main.merge(members, how = 'left', left_on = 'merge_key', right_on='people')

,Unnamed: 0,transaction_date,ticker,asset_description,type,amount,representative,merge_key,people,state,party,total_senate_years,total_house_years,total_years
0,4,2020-03-18,CBRE,CBRE Group Inc Common Stock Class A,sale_partial,"$1,001 - $15,000",Susie Lee,Susie Lee,Susie Lee,Nevada,Democratic,0,2,2
1,5,2020-03-24,CNC,Centene Corporation,purchase,"$1,001 - $15,000",Susie Lee,Susie Lee,Susie Lee,Nevada,Democratic,0,2,2
2,6,2020-03-31,DG,Dollar General Corporation,purchase,"$1,001 - $15,000",Susie Lee,Susie Lee,Susie Lee,Nevada,Democratic,0,2,2
3,10,2020-02-21,ENV,"Envestnet, Inc",sale_partial,"$1,001 - $15,000",Susie Lee,Susie Lee,Susie Lee,Nevada,Democratic,0,2,2
4,12,2020-03-17,LKQ,LKQ Corporation,sale_partial,"$1,001 - $15,000",Susie Lee,Susie Lee,Susie Lee,Nevada,Democratic,0,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13218,8319,2014-01-29,C,"Citigroup, Inc. (NYSE)",sale_partial,"$1,001 - $15,000",Susan M Collins,"Susan Collins, M.","Susan Collins, M.",Maine,Republican,24,0,24
13219,8320,2014-01-28,EBAY,eBay Inc. (NASDAQ),sale_partial,"$1,001 - $15,000",Susan M Collins,"Susan Collins, M.","Susan Collins, M.",Maine,Republican,24,0,24
13220,8321,2014-01-24,FB,"Facebook, Inc. (NASDAQ)",purchase,"$1,001 - $15,000",Susan M Collins,"Susan Collins, M.","Susan Collins, M.",Maine,Republican,24,0,24
13221,8322,2014-01-24,CRM,"Salesforce.com, Inc (NYSE)",purchase,"$1,001 - $15,000",Susan M Collins,"Susan Collins, M.","Susan Collins, M.",Maine,Republican,24,0,24


In [104]:
main.to_csv('./data/megan_senate_house_merged_years_served.csv', index=False)